In [3]:
import torch
import torch.nn as nn
import torch.optim
import random
import numpy as np
import spacy
import datasets
import torchtext

In [4]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
dataset = datasets.load_dataset("bentrevett/multi30k")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [7]:
train_data, valid_data, test_data = dataset['train'], dataset['validation'], dataset['test']

In [8]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [9]:
string = "What a lovel day it is today!"
[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovel', 'day', 'it', 'is', 'today', '!']

In [10]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

In [12]:
max_length = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {"en_nlp": en_nlp, "de_nlp": de_nlp, "max_length": max_length, "lower": lower, "sos_token": sos_token, "eos_token": eos_token}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 8779.69 examples/s]


In [15]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer


min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens =[
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab = build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)


de_vocab = build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

In [16]:
en_vocab.get_itos()[:100]


['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'a',
 '.',
 'in',
 'the',
 'on',
 'man',
 'is',
 'and',
 'of',
 'with',
 'woman',
 ',',
 'two',
 'are',
 'to',
 'people',
 'at',
 'an',
 'wearing',
 'shirt',
 'young',
 'white',
 'black',
 'his',
 'while',
 'blue',
 'men',
 'red',
 'sitting',
 'girl',
 'boy',
 'dog',
 'standing',
 'playing',
 'group',
 'street',
 'down',
 'walking',
 '-',
 'front',
 'her',
 'holding',
 'one',
 'water',
 'three',
 'by',
 'women',
 'up',
 'green',
 'little',
 'for',
 'child',
 'looking',
 'outside',
 'as',
 'large',
 'through',
 'brown',
 'yellow',
 'children',
 'person',
 'from',
 'their',
 'hat',
 'ball',
 'into',
 'small',
 'next',
 'other',
 'dressed',
 'some',
 'out',
 'over',
 'building',
 'riding',
 'running',
 'near',
 'jacket',
 'another',
 'around',
 'sidewalk',
 'field',
 'orange',
 'crowd',
 'beach',
 'stands',
 'pink',
 'sits',
 'jumping',
 'behind',
 'table',
 'snow',
 'grass',
 'hair',
 'background',
 'bike']

In [17]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [18]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [19]:
en_vocab.get_itos()[0]

'<unk>'